In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import numpy as np
import mapping
from tensorflow.keras.preprocessing import image
import splitfolders
import matplotlib.pyplot as plt
from tensorflow.keras.utils import register_keras_serializable

In [2]:
label_map_gender = {
    "male": "ذكر",
    "female": "أنثى",
}
label_map_religion= {
    "muslim_m": "مسلم",
    "muslim_f": "مسلمة",
    "christian_m": "مسيحي",
    "christian_f": "مسيحية"
}
label_map_marital_status={
    "single_m": "أعزب",
    "single_f": "أنسة",
    "married_m": "متزوج",
    "married_f": "متزوجة",
    "widow": "أرملة",
    "widower": "أرمل",
}

In [4]:
img_size = (64, 64)
batch_size = 32

# train_ds = tf.keras.utils.image_dataset_from_directory(
#     r"..\data\dataset_cnn_2\data\train",
#     seed=123,
#     image_size=img_size,
#     shuffle=True,
#     batch_size=batch_size,
#     color_mode="rgb"  
# )

# val_ds = tf.keras.utils.image_dataset_from_directory(
#     r"..\data\dataset_cnn_2\data\val",
#     seed=123,
#     shuffle=True,
#     image_size=img_size,
#     batch_size=batch_size,
#     color_mode="rgb"
# )

# test_ds = tf.keras.utils.image_dataset_from_directory(
#     r"..\data\dataset_cnn_2\data\test",
#     seed=123,
#     shuffle=True,
#     image_size=img_size,
#     batch_size=batch_size,
#     color_mode="rgb"
# )

gender_train_ds = tf.keras.utils.image_dataset_from_directory(
    r"..\data\dataset_cnn_2\data\gender\train",
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical"
)

gender_val_ds = tf.keras.utils.image_dataset_from_directory(
    r"..\data\dataset_cnn_2\data\gender\val",
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical"
)

gender_test_ds = tf.keras.utils.image_dataset_from_directory(
    r"..\data\dataset_cnn_2\data\gender\test",
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical"
)

# Normalization layer
normalization_layer = tf.keras.layers.Rescaling(1./255)

class_names_gender = gender_train_ds.class_names
print("Classes:", class_names_gender)
num_classes = len(class_names_gender)

gender_train_ds = gender_train_ds.map(lambda x, y: (normalization_layer(x), y))
gender_val_ds = gender_val_ds.map(lambda x, y: (normalization_layer(x), y))
gender_test_ds = gender_test_ds.map(lambda x, y: (normalization_layer(x), y))

Found 2884 files belonging to 2 classes.
Found 780 files belonging to 2 classes.
Found 457 files belonging to 2 classes.
Classes: ['female', 'male']


In [6]:

# # class MinPooling2D(layers.Layer):
# #     def __init__(self, pool_size=(2, 2), strides=None, padding="VALID", **kwargs):
# #         super(MinPooling2D, self).__init__(**kwargs)
# #         self.pool_size = pool_size
# #         self.strides = strides if strides is not None else pool_size
# #         self.padding = padding

# #     def call(self, inputs):
# #         return -tf.nn.max_pool2d(
# #             -inputs,
# #             ksize=self.pool_size,
# #             strides=self.strides,
# #             padding=self.padding
# #         )

# import tensorflow as tf
# from keras import layers
# from keras.saving import register_keras_serializable

# @register_keras_serializable(package="Custom")
# class MinPooling2D(layers.Layer):
#     def __init__(self, pool_size=(2, 2), strides=None, padding="VALID", **kwargs):
#         super().__init__(**kwargs)
#         self.pool_size = tuple(pool_size)
#         self.strides = tuple(strides) if strides is not None else tuple(pool_size)
#         self.padding = padding.upper()

#     def call(self, inputs):
#         return -tf.nn.max_pool2d(
#             -inputs,
#             ksize=self.pool_size,
#             strides=self.strides,
#             padding=self.padding
#         )

#     def get_config(self):
#         config = super().get_config()
#         config.update({
#             "pool_size": self.pool_size,
#             "strides": self.strides,
#             "padding": self.padding
#         })
#         return config


In [5]:
from custom_layers.MinPooling import MinPooling2D

In [10]:
model = models.Sequential([
    #layers.Conv2D(16, 3, activation='relu', input_shape=(64, 64, 3)),
    #layers.MaxPooling2D(),
    
    layers.Conv2D(32, 3, activation='relu'),
    MinPooling2D(pool_size=(2,2)),
    #layers.MaxPooling2D(),
    #layers.AveragePooling2D(pool_size=(2,2)),

    layers.Conv2D(64, 3, activation='relu'),
    #layers.MaxPooling2D(),
    
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(256, 3, activation='relu'),
    #layers.AveragePooling2D(pool_size=(2,2)),
    MinPooling2D(pool_size=(2,2)),
    
    
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [11]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
h = model.fit(gender_train_ds, epochs=40, 
                    validation_data=gender_val_ds)

Epoch 1/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 63s 480ms/step - accuracy: 0.8499 - loss: 0.2805 - val_accuracy: 0.9987 - val_loss: 0.0065
Epoch 2/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 42s 455ms/step - accuracy: 0.9951 - loss: 0.0192 - val_accuracy: 0.9949 - val_loss: 0.0159
Epoch 3/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 45s 487ms/step - accuracy: 0.9976 - loss: 0.0117 - val_accuracy: 0.9923 - val_loss: 0.0169
Epoch 4/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 41s 443ms/step - accuracy: 0.9972 - loss: 0.0110 - val_accuracy: 1.0000 - val_loss: 0.0017
Epoch 5/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 27s 293ms/step - accuracy: 0.9927 - loss: 0.0231 - val_accuracy: 0.9167 - val_loss: 0.2036
Epoch 6/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 55s 607ms/step - accuracy: 0.9893 - loss: 0.0410 - val_accuracy: 0.9974 - val_loss: 0.0033
Epoch 7/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 57s 619ms/step - accuracy: 0.9969 - loss: 0.0096 - val_accuracy: 0.9987 - val_loss: 0.0029
Epoch 8/40
91/91 ━━━━━━━━━━━━━━━━━━━━ 47s 516ms/step - accuracy: 0.9976 - loss: 0.0054 - val_accu

In [13]:
test_loss, test_acc = model.evaluate(gender_test_ds)

15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 136ms/step - accuracy: 0.9978 - loss: 0.0182


In [21]:
folder_path = r"..\notebooks\outputs\new"
#D:\ocr_project\data\dataset_cnn_2\output_img\result\crops\gender\male
#religion\muslim_m   "D:\ocr_project\data\dataset_cnn_2\output_img\result\crops\gender\male"
#marital status\widower   "..\data\dataset_cnn_2\data\test\male"
output_file = "../outputs/predictions_CNN_muslim.txt"

with open(output_file, "w", encoding="utf-8") as f:
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)

       
        if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

      
        img = image.load_img(img_path, target_size=(64,64))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0

     
        predictions = model.predict(img_array, verbose=0)
        predicted_label = np.argmax(predictions, axis=1)[0]

        en_label = class_names[predicted_label]
        ar_label = label_map_gender.get(en_label, en_label)

     
        f.write(f"{img_name}: {ar_label}\n")

print(f"{output_file}")

../outputs/predictions_CNN_muslim.txt


In [18]:
model.save("../models/cnn_model_gender.keras")